In [3]:
import pandas as pd
import warnings
import tkinter as tk
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24

fecha = f"{año}{str(mes).zfill(2)}" # 202401

reporte_condonacion = f"Reporte de Condonación 10.10.2024.xlsx" # Modificar
base_final = f"Base_Final_{fecha}.xlsx"
base_final_path = os.path.abspath(base_final)

print("Reporte de Condonación:", reporte_condonacion)
print("Base Final:", base_final_path)

Reporte de Condonación: Reporte de Condonación 10.10.2024.xlsx
Base Final: c:\Users\miria\Documents\BBVA\GitHub\condonaciones\Base_Final_202410.xlsx


In [4]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [5]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar reporte de condonación?", icon='warning')
if result == 'yes':
    df_base = pd.read_excel(reporte_condonacion)

In [99]:
df_base_test = df_base.copy()
print(df_base_test.shape)

df_base_test.columns = clean_columns(df_base_test.columns)

cols_required = [
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE', 
    'CC', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA', 
    'MARCA_TEMPORAL', 
    'FECHA_DE_REVISIÓN', 
    '¿REVISIÓN_CONFORME?', 
    'FECHA_DE_EJECUCIÓN', 
    'EJECUCIÓN'
    ]

df_base_test = df_base_test[cols_required]
df_base_test.rename(columns={
    'MARCA_TEMPORAL': 'FECHA_INGRESO', 
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE': 'AGENCIA', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA': 'CONTRATO', 
    'FECHA_DE_EJECUCIÓN': 'FECHA_EJECUCION', 
    '¿REVISIÓN_CONFORME?': 'CONFORME', 
    'FECHA_DE_REVISIÓN': 'FECHA_REVISION', 
    'EJECUCIÓN': 'EJECUCION'
    }, inplace=True)

df_base_test['AGENCIA'] = df_base_test['AGENCIA'].str.upper()
df_base_test['EJECUCION'] = df_base_test['EJECUCION'].str.upper()
df_base_test['CONFORME'] = df_base_test['CONFORME'].str.upper()
df_base_test['CC'] = df_base_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].astype('Int64').astype(str).str.zfill(18)

df_base_test['FECHA_INGRESO'] = pd.to_datetime(df_base_test['FECHA_INGRESO'], errors='coerce')
df_base_test['FECHA_REVISION'] = pd.to_datetime(df_base_test['FECHA_REVISION'], errors='coerce')
df_base_test['FECHA_EJECUCION'] = pd.to_datetime(df_base_test['FECHA_EJECUCION'], errors='coerce')

df_base_test['FECHA_INGRESO'] = df_base_test['FECHA_INGRESO'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_base_test['FECHA_REVISION'] = df_base_test['FECHA_REVISION'].dt.strftime('%d-%m-%Y')
df_base_test['FECHA_EJECUCION'] = df_base_test['FECHA_EJECUCION'].dt.strftime('%d-%m-%Y')

df_base_test = df_base_test[df_base_test['AGENCIA'].isin(['RJ ABOGADOS', 'CLASA', 'MORNESE'])]
df_base_test['AGENCIA'] = df_base_test['AGENCIA'].replace('RJ ABOGADOS', 'RJ')

df_base_test = df_base_test[df_base_test['FECHA_REVISION'].notnull()]
df_base_test = df_base_test[~((df_base_test['EJECUCION'] == 'SI') & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].isnull()) & (df_base_test['FECHA_EJECUCION'].notnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].notnull()) & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'NO') & (df_base_test['EJECUCION'] == 'SI'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'] == 'NO'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'].isnull()))]

df_base_test.sort_values(by=['CONTRATO', 'FECHA_INGRESO'], ascending=True, inplace=True)
df_base_test.reset_index(drop=True, inplace=True)

print(df_base_test.shape)
df_base_test.head(5)

(55171, 13)
(42562, 8)


,AGENCIA,CC,CONTRATO,FECHA_INGRESO,FECHA_REVISION,CONFORME,FECHA_EJECUCION,EJECUCION
0,MORNESE,26910745,000000000000755211,04-06-2024 10:33:32,11-06-2024,NO,NaN,NaN
1,RJ,24204625,001100025000043500,07-09-2023 21:43:55,08-09-2023,SI,08-09-2023,SI
2,MORNESE,26217657,001100025000080309,24-07-2023 13:37:41,26-07-2023,SI,26-07-2023,SI
3,CLASA,27608713,001100025000130535,31-03-2023 13:22:46,03-04-2023,SI,03-04-2023,SI
4,MORNESE,24007029,001100025000163735,02-08-2024 13:19:09,14-08-2024,SI,14-08-2024,SI


In [102]:
df_base_test[['CONFORME', 'EJECUCION']].value_counts(dropna=False)


CONFORME  EJECUCION
SI        SI           40124
NO        NaN           2438
Name: count, dtype: int64